In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import fetch_20newsgroups
import re
import string

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Libraries imported successfully!")


In [ ]:
# Implementing Many-to-One RNN for Sentiment Analysis
def create_many_to_one_rnn(vocab_size, embedding_dim=100, rnn_units=64, max_length=100):
    """
    Create a Many-to-One RNN for text classification
    """
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.LSTM(rnn_units, return_sequences=False, dropout=0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create sample text data for sentiment analysis
def create_sentiment_data(n_samples=1000):
    """Create synthetic sentiment data"""
    positive_words = ['good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic', 'love', 'perfect']
    negative_words = ['bad', 'terrible', 'awful', 'hate', 'horrible', 'disgusting', 'worst', 'pathetic']
    neutral_words = ['the', 'and', 'is', 'in', 'to', 'of', 'it', 'that', 'have', 'for']
    
    texts = []
    labels = []
    
    for _ in range(n_samples):
        # Randomly choose sentiment
        sentiment = np.random.choice([0, 1])  # 0: negative, 1: positive
        
        if sentiment == 1:
            # Create positive text
            words = np.random.choice(positive_words, size=np.random.randint(3, 8)).tolist()
            words += np.random.choice(neutral_words, size=np.random.randint(2, 5)).tolist()
        else:
            # Create negative text
            words = np.random.choice(negative_words, size=np.random.randint(3, 8)).tolist()
            words += np.random.choice(neutral_words, size=np.random.randint(2, 5)).tolist()
        
        np.random.shuffle(words)
        text = ' '.join(words)
        
        texts.append(text)
        labels.append(sentiment)
    
    return texts, np.array(labels)

# Generate sentiment data
print("Creating sentiment analysis dataset...")
texts, labels = create_sentiment_data(2000)

print(f"Sample texts:")
for i in range(3):
    print(f"Text: '{texts[i]}' -> Label: {labels[i]} ({'Positive' if labels[i] else 'Negative'})")

# Tokenization and preprocessing
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
max_length = 20
padded_sequences = keras.preprocessing.sequence.pad_sequences(
    sequences, maxlen=max_length, truncating='post'
)

print(f"Vocabulary size: {len(tokenizer.word_index)}")
print(f"Sequence shape: {padded_sequences.shape}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42
)

# Create and train Many-to-One model
print("\nCreating Many-to-One RNN model...")
model_many_to_one = create_many_to_one_rnn(
    vocab_size=len(tokenizer.word_index) + 1,
    embedding_dim=50,
    rnn_units=32,
    max_length=max_length
)

print("Model architecture:")
model_many_to_one.summary()

print("\nTraining Many-to-One model...")
history = model_many_to_one.fit(
    X_train, y_train,
    batch_size=32,
    epochs=10,
    validation_data=(X_test, y_test),
    verbose=1
)


In [ ]:
# Implementing Many-to-Many RNN (Synchronized) for Sequence Tagging
def create_many_to_many_rnn(vocab_size, tag_size, embedding_dim=50, rnn_units=64, max_length=20):
    """
    Create a Many-to-Many RNN for sequence tagging
    """
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(tag_size, activation='softmax'))
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create synthetic POS tagging data
def create_pos_tagging_data(n_samples=1000):
    """Create synthetic POS tagging data"""
    words = ['the', 'cat', 'dog', 'runs', 'jumps', 'quickly', 'slowly', 'big', 'small', 'red']
    pos_tags = {
        'the': 0,      # DET
        'cat': 1,      # NOUN
        'dog': 1,      # NOUN
        'runs': 2,     # VERB
        'jumps': 2,    # VERB
        'quickly': 3,  # ADV
        'slowly': 3,   # ADV
        'big': 4,      # ADJ
        'small': 4,    # ADJ
        'red': 4       # ADJ
    }
    
    sequences = []
    tag_sequences = []
    
    for _ in range(n_samples):
        seq_length = np.random.randint(3, 8)
        sequence = np.random.choice(words, size=seq_length).tolist()
        tags = [pos_tags[word] for word in sequence]
        
        sequences.append(sequence)
        tag_sequences.append(tags)
    
    return sequences, tag_sequences, pos_tags

print("Creating POS tagging dataset...")
sequences, tag_sequences, pos_dict = create_pos_tagging_data(1500)

print(f"Sample sequences:")
for i in range(3):
    print(f"Words: {sequences[i]}")
    print(f"Tags:  {tag_sequences[i]}")
    print()

# Create word-to-index mapping
word_to_idx = {word: idx for idx, word in enumerate(set([word for seq in sequences for word in seq]))}
word_to_idx['<PAD>'] = len(word_to_idx)

# Convert to numerical sequences
max_seq_length = 10
X_sequences = []
y_sequences = []

for seq, tags in zip(sequences, tag_sequences):
    # Convert words to indices
    x_seq = [word_to_idx[word] for word in seq]
    y_seq = tags
    
    # Pad sequences
    if len(x_seq) < max_seq_length:
        x_seq.extend([word_to_idx['<PAD>']] * (max_seq_length - len(x_seq)))
        y_seq.extend([0] * (max_seq_length - len(y_seq)))  # Pad with DET tag
    else:
        x_seq = x_seq[:max_seq_length]
        y_seq = y_seq[:max_seq_length]
    
    X_sequences.append(x_seq)
    y_sequences.append(y_seq)

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

print(f"Sequence data shape: {X_sequences.shape}")
print(f"Tag data shape: {y_sequences.shape}")

# Split data
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    X_sequences, y_sequences, test_size=0.2, random_state=42
)

# Create and train Many-to-Many model
print("\nCreating Many-to-Many RNN model...")
model_many_to_many = create_many_to_many_rnn(
    vocab_size=len(word_to_idx),
    tag_size=5,  # Number of POS tags
    embedding_dim=30,
    rnn_units=32,
    max_length=max_seq_length
)

print("Model architecture:")
model_many_to_many.summary()

print("\nTraining Many-to-Many model...")
history_seq = model_many_to_many.fit(
    X_train_seq, y_train_seq,
    batch_size=32,
    epochs=15,
    validation_data=(X_test_seq, y_test_seq),
    verbose=1
)


In [ ]:
# CNN vs RNN Comparison for Text Classification
def create_cnn_model(vocab_size, embedding_dim=50, max_length=20):
    """Create a CNN model for text classification"""
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.Conv1D(128, 5, activation='relu'),
        keras.layers.GlobalMaxPooling1D(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create CNN model for comparison
print("Creating CNN model for comparison...")
cnn_model = create_cnn_model(
    vocab_size=len(tokenizer.word_index) + 1,
    embedding_dim=50,
    max_length=max_length
)

print("CNN Model architecture:")
cnn_model.summary()

# Train CNN model
print("\nTraining CNN model...")
import time

start_time = time.time()
cnn_history = cnn_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=10,
    validation_data=(X_test, y_test),
    verbose=0
)
cnn_training_time = time.time() - start_time

# Compare performance
rnn_accuracy = model_many_to_one.evaluate(X_test, y_test, verbose=0)[1]
cnn_accuracy = cnn_model.evaluate(X_test, y_test, verbose=0)[1]

print(f"\nPerformance Comparison:")
print(f"RNN Accuracy: {rnn_accuracy:.4f}")
print(f"CNN Accuracy: {cnn_accuracy:.4f}")
print(f"CNN Training Time: {cnn_training_time:.2f} seconds")

# Model parameter comparison
rnn_params = model_many_to_one.count_params()
cnn_params = cnn_model.count_params()

print(f"\nModel Complexity:")
print(f"RNN Parameters: {rnn_params:,}")
print(f"CNN Parameters: {cnn_params:,}")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy comparison
models = ['RNN (LSTM)', 'CNN']
accuracies = [rnn_accuracy, cnn_accuracy]
colors = ['lightcoral', 'lightblue']

axes[0, 0].bar(models, accuracies, color=colors)
axes[0, 0].set_title('Model Accuracy Comparison')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_ylim(0.5, 1.0)
for i, v in enumerate(accuracies):
    axes[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center')

# Parameter comparison
param_counts = [rnn_params, cnn_params]
axes[0, 1].bar(models, param_counts, color=colors)
axes[0, 1].set_title('Model Complexity (Parameters)')
axes[0, 1].set_ylabel('Number of Parameters')
for i, v in enumerate(param_counts):
    axes[0, 1].text(i, v + 1000, f'{v:,}', ha='center')

# Training curves
axes[1, 0].plot(history.history['val_accuracy'], label='RNN', linewidth=2)
axes[1, 0].plot(cnn_history.history['val_accuracy'], label='CNN', linewidth=2)
axes[1, 0].set_title('Validation Accuracy During Training')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Validation Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Loss curves
axes[1, 1].plot(history.history['val_loss'], label='RNN', linewidth=2)
axes[1, 1].plot(cnn_history.history['val_loss'], label='CNN', linewidth=2)
axes[1, 1].set_title('Validation Loss During Training')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Validation Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("1. CNNs often train faster due to parallel processing")
print("2. RNNs better capture sequential dependencies")
print("3. Choice depends on the specific task requirements")
print("4. For short texts, CNNs might be sufficient")
print("5. For long sequences with complex dependencies, RNNs excel")
